In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras import layers
from vacances_scolaires_france import SchoolHolidayDates
from jours_feries_france import JoursFeries
from datetime import datetime, date
import seaborn as sns
import matplotlib.pyplot as plt
from astral import LocationInfo
from astral.sun import daylight
from sklearn.kernel_ridge import KernelRidge

In [3]:
data = pd.read_parquet('../data/train.parquet')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455163 entries, 48321 to 928462
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   counter_id                 455163 non-null  category      
 1   counter_name               455163 non-null  category      
 2   site_id                    455163 non-null  int64         
 3   site_name                  455163 non-null  category      
 4   bike_count                 455163 non-null  float64       
 5   date                       455163 non-null  datetime64[ns]
 6   counter_installation_date  455163 non-null  datetime64[ns]
 7   counter_technical_id       455163 non-null  category      
 8   latitude                   455163 non-null  float64       
 9   longitude                  455163 non-null  float64       
 10  log_bike_count             455163 non-null  float64       
dtypes: category(4), datetime64[ns](2), float64(4), i

In [6]:
data.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,log_bike_count
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.000000
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.693147
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.000000
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,1.609438
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,2.302585


In [9]:
d = SchoolHolidayDates()
jf = JoursFeries()
data['hour'] = data.date.dt.hour
data['weekday'] = data.date.dt.weekday
data['dom'] = data.date.dt.day
data['week'] = data.date.dt.isocalendar().week
data['month'] = data.date.dt.month
data['year'] = data.date.dt.year
data['dom_counter'] = data.counter_installation_date.dt.day
data['month_counter'] = data.counter_installation_date.dt.month
data['year_counter'] = data.counter_installation_date.dt.year
data['date_datetime'] = data.date.map(lambda x: x.to_pydatetime().date())
data['is_ferie'] = data.date_datetime.map(lambda x: jf.is_bank_holiday(x, zone='Métropole'))
data['is_holiday'] = data.date_datetime.map(lambda x: d.is_holiday_for_zone(x, 'C'))

In [10]:
def is_daylight(x):
    city=LocationInfo('Paris', timezone='Europe/Paris')
    sun_info = daylight(city.observer, date=x.to_pydatetime().date(), tzinfo='Europe/Paris')
    x = x.tz_localize('Europe/Paris', ambiguous=True, nonexistent='shift_forward')
    return (x > sun_info[0]) & (x < sun_info[1])

In [11]:
data['is_daylight'] = data.date.map(is_daylight)

In [12]:
data.drop(columns=['counter_name', 'site_name', 'counter_technical_id', 'counter_installation_date'], inplace=True)

In [54]:
external_data = pd.read_csv("external_data_reworked.csv", parse_dates=['date'])

In [55]:
data

,counter_id,site_id,bike_count,date,latitude,longitude,log_bike_count,hour,weekday,dom,week,month,year,dom_counter,month_counter,year_counter,date_datetime,is_ferie,is_holiday,is_daylight
48321,100007049-102007049,100007049,0.0,2020-09-01 02:00:00,48.846028,2.375429,0.000000,2,1,1,36,9,2020,18,1,2013,2020-09-01,False,False,False
48324,100007049-102007049,100007049,1.0,2020-09-01 03:00:00,48.846028,2.375429,0.693147,3,1,1,36,9,2020,18,1,2013,2020-09-01,False,False,False
48327,100007049-102007049,100007049,0.0,2020-09-01 04:00:00,48.846028,2.375429,0.000000,4,1,1,36,9,2020,18,1,2013,2020-09-01,False,False,False
48330,100007049-102007049,100007049,4.0,2020-09-01 15:00:00,48.846028,2.375429,1.609438,15,1,1,36,9,2020,18,1,2013,2020-09-01,False,False,True
48333,100007049-102007049,100007049,9.0,2020-09-01 18:00:00,48.846028,2.375429,2.302585,18,1,1,36,9,2020,18,1,2013,2020-09-01,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928450,300014702-353245971,300014702,51.0,2021-08-08 18:00:00,48.839770,2.301980,3.951244,18,6,8,31,8,2021,29,11,2020,2021-08-08,False,True,True
928453,300014702-353245971,300014702,1.0,2021-08-09 02:00:00,48.839770,2.301980,0.693147,2,0,9,32,8,2021,29,11,2020,2021-08-09,False,True,False
928456,300014702-353245971,300014702,61.0,2021-08-09 08:00:00,48.839770,2.301980,4.127134,8,0,9,32,8,2021,29,11,2020,2021-08-09,False,True,True
928459,300014702-353245971,300014702,44.0,2021-08-09 10:00:00,48.839770,2.301980,3.806662,10,0,9,32,8,2021,29,11,2020,2021-08-09,False,True,True


In [56]:
external_data

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,100057380-104057380,100057380-103057380,100057445-103057445,100057445-104057445,100060178-102060178,100060178-101060178,100063175-353277233,100063175-353277235,300014702-353245972,300014702-353245971
0,7149,2021-01-01 00:00:00,100810,80,1,270,1.8,272.75,272.15,96,...,False,False,False,False,False,False,False,False,False,False
1,7149,2021-01-01 03:00:00,100920,110,3,300,1.7,271.25,270.95,98,...,False,False,False,False,False,False,False,False,False,False
2,7149,2021-01-01 06:00:00,100950,30,3,290,2.6,271.95,271.65,98,...,False,False,False,False,False,False,False,False,False,False
3,7149,2021-01-01 09:00:00,101100,150,2,280,1.7,272.45,272.05,97,...,False,False,False,False,False,False,False,False,False,False
4,7149,2021-01-01 12:00:00,101110,30,0,50,1.0,276.95,274.15,82,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,7149,2020-09-30 09:00:00,101540,-30,8,230,4.4,289.95,286.85,82,...,False,False,False,False,False,False,False,False,True,True
3318,7149,2020-09-30 12:00:00,101320,-210,8,190,4.9,292.05,285.55,66,...,False,False,False,False,False,False,False,False,True,True
3319,7149,2020-09-30 15:00:00,101140,-180,7,190,4.1,291.55,286.45,72,...,False,False,False,False,False,False,False,False,True,True
3320,7149,2020-09-30 18:00:00,101020,-130,6,190,2.7,290.15,285.25,73,...,False,False,False,False,False,False,False,False,True,True


In [57]:
external_data_grouped = external_data.groupby(by=['date_datetime']).sum()

In [58]:
external_data_grouped

,numer_sta,pmer,tend,cod_tend,dd,ff,t,td,u,vv,...,100057380-104057380,100057380-103057380,100057445-103057445,100057445-104057445,100060178-102060178,100060178-101060178,100063175-353277233,100063175-353277235,300014702-353245972,300014702-353245971
date_datetime,,,,,,,,,,,,,,,,,,,,,
2020-09-01,57192,815240,-160,40,1460,19.7,2314.30,2252.30,505,210830,...,0,0,0,0,0,0,0,0,8,8
2020-09-02,57192,815730,180,32,2240,16.0,2321.90,2247.20,452,257580,...,0,0,0,0,0,0,0,0,8,8
2020-09-03,57192,816720,150,30,1800,24.8,2359.10,2271.60,409,274880,...,0,0,0,0,0,0,0,0,8,8
2020-09-04,57192,817220,-180,35,1970,23.5,2367.00,2286.70,453,280410,...,0,0,0,0,0,0,0,0,8,8
2020-09-05,57192,817230,240,26,2600,33.6,2328.00,2269.10,510,243590,...,0,0,0,0,0,0,0,0,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-17,57192,816460,-30,38,610,14.4,2265.70,2231.60,614,158000,...,0,0,0,0,0,0,0,0,0,0
2021-10-18,57192,817320,60,30,1200,18.5,2281.20,2262.70,692,157350,...,0,0,0,0,0,0,0,0,0,0
2021-10-19,57192,815710,-430,39,1580,29.7,2333.50,2286.30,564,157310,...,0,0,0,0,0,0,0,0,0,0


In [59]:
counters_list = data.counter_id.unique()

In [60]:
down_counters = external_data_grouped[counters_list] > 0

In [70]:
days_down={}
for counter in counters_list :
    string_list = down_counters[down_counters[counter] == True].index.values.tolist()
    date_datetime_list = [pd.to_datetime(j).date() for j in string_list]
    days_down[counter] = date_datetime_list

In [71]:
days_down

{'100007049-102007049': [],
 '100007049-101007049': [],
 '100036718-104036718': [],
 '100036718-103036718': [],
 '100036719-104036719': [],
 '100036719-103036719': [],
 '100042374-110042374': [datetime.date(2021, 3, 13),
  datetime.date(2021, 3, 14),
  datetime.date(2021, 3, 15),
  datetime.date(2021, 3, 16),
  datetime.date(2021, 3, 17),
  datetime.date(2021, 3, 18),
  datetime.date(2021, 3, 19),
  datetime.date(2021, 3, 20),
  datetime.date(2021, 3, 21),
  datetime.date(2021, 3, 22),
  datetime.date(2021, 3, 23),
  datetime.date(2021, 3, 24),
  datetime.date(2021, 3, 25),
  datetime.date(2021, 3, 26),
  datetime.date(2021, 3, 27),
  datetime.date(2021, 3, 28),
  datetime.date(2021, 3, 29),
  datetime.date(2021, 3, 30)],
 '100042374-109042374': [datetime.date(2021, 3, 13),
  datetime.date(2021, 3, 14),
  datetime.date(2021, 3, 15),
  datetime.date(2021, 3, 16),
  datetime.date(2021, 3, 17),
  datetime.date(2021, 3, 18),
  datetime.date(2021, 3, 19),
  datetime.date(2021, 3, 20),
  dat

In [72]:
data['is_down'] = data.apply(lambda x: x.date_datetime in days_down[x.counter_id], axis=1)

In [73]:
data['is_down'].sum()

5208